## Library Imports

In [47]:
import numpy as np  
import matplotlib.pyplot as plt  # Plotting
import seaborn as sns  # Better visualisation
import pandas as pd  
import datetime  # Date and time operations
from datetime import datetime 
from sklearn.metrics import mean_squared_error  
from sklearn.preprocessing import StandardScaler  # Feature scaling
from keras.models import load_model  
from binance.client import Client  # Binance API interaction
from binance.exceptions import BinanceAPIException, BinanceOrderException  # API exceptions
import time  

# Rate limiter setup
def rate_limiter(max_per_second):
    """
    Decorator function to limit the rate of API calls.

    Args:
        max_per_second (int): Maximum number of API calls allowed per second.

    Returns:
        function: Decorated function with rate limiting.
    """
    min_interval = 1.0 / float(max_per_second)
    def decorate(func):
        last_called = [0.0]
        def rate_limited_function(*args, **kargs):
            elapsed = time.time() - last_called[0]
            left_to_wait = min_interval - elapsed
            if left_to_wait > 0:
                time.sleep(left_to_wait)
            last_called[0] = time.time()
            return func(*args, **kargs)
        return rate_limited_function
    return decorate

The framework for data analysis and visualisation is built up by this code, which also includes libraries for managing time-related activities, graphing and numerical computation. It also provides a rate limiter function that controls the number of times an API call is made while using the Binance API.

## Binance API Setup

In [48]:
# Define Binance API keys
api_key = ""    # Binance API key
secret_key = ""  # Binance API secret key

# Binance client with error handling
try:
    client = Client(api_key, secret_key)
    print("Client Loaded Successfully")
except Exception as e:
    print(f"Error initialising Binance Client: {e}")

Client Loaded Successfully


## Fetch the Data

In [49]:
# Fetch candlestick data from Binance with error handling
symbol = 'BTCUSDT'  # Symbol for Bitcoin against USDT
interval = Client.KLINE_INTERVAL_15MINUTE  # Interval for candlestick data
limit = 1000  # Number of data points to fetch

try:
    dataset = client.get_klines(symbol=symbol, interval=interval, limit=limit)
    print("Data fetch successful")
except BinanceAPIException as e:
    print(f"API Exception occurred: {e.status_code} - {e.message}")
except BinanceOrderException as e:
    print(f"Order Exception occurred: {e.status_code} - {e.message}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data fetch successful


## Preprocessing the Data


In [50]:
# Preprocess the data
price = np.array([float(dataset[i][4]) for i in range(len(dataset))])
price = price.reshape(-1, 1) 

# StandardScaler
scaler = StandardScaler()
scaler.fit(price[:800])  # Fit the scaler on the first 800 candles for training
prices_scaled = scaler.transform(price)  # Transform the entire dataset

StandardScaler is used to standardise the data in order to guarantee consistency and get the data ready for live trading. 

## Loading the Model

In [51]:
# Load LSTM model
model_path = "Final_LSTM%.h5"
loaded_model = load_model(model_path)
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 4, 256)           133120    
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 4, 256)            0         
                                                                 
 lstm_4 (LSTM)               (None, 4, 64)             82176     
                                                                 
 dropout_3 (Dropout)         (None, 4, 64)             0         
                                                                 
 lstm_5 (LSTM)               (None, 32)                12416     
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                      

In [52]:
index = [996, 997, 998, 999]
candles = scaler.transform(np.array([float(dataset[i][4]) for i in index]).reshape(-1,1))

model_feed = candles.reshape(1,4,1)
# (1, 4, 1) for prediction
prediction = loaded_model.predict(model_feed)

# We reshape it to (-1, 1) to make it a 2D array with a single column
prediction_reshaped = prediction.reshape(-1, 1)

# Now we can apply the inverse transform
predicted_price = scaler.inverse_transform(prediction_reshaped)[0][0]

print(predicted_price)

1/1 [==============================] - 2s 2s/step
69460.42


A loaded model is used  to estimate the future price after reshaping the scaled candle data to the appropriate format. To get the predicted cost in the original scale, it then performs the inverse transform.

## Risk Management

In [53]:
# Define stop loss and take profit percentages
stop_loss_percentage = 0.05  # 5%
take_profit_percentage = 0.1  # 10%

account_balance = 20  # Based on my actual account balance
risk_per_trade = 0.02

In [54]:
# Define a function to calculate the position size based on account balance, risk per trade and stop loss percentage
def calculate_position_size(account_balance, risk_per_trade, stop_loss_percentage):
    """
    Calculates the position size based on the account balance, risk per trade and stop loss percentage.

    """
    risk_amount = account_balance * risk_per_trade
    position_size = risk_amount / (stop_loss_percentage * account_balance)
    return position_size

# Calculate the position size
position_size = calculate_position_size(account_balance, risk_per_trade, stop_loss_percentage)

# Print the calculated position size
print(f"Position size based on account balance and risk per trade: {position_size}")


Position size based on account balance and risk per trade: 0.4


In [55]:
# Apply the rate limiter to control API calls
@rate_limiter(1)
def get_current_price(symbol):
    # Get the current price of a symbol from the Binance API.
    try:
        current_prices = client.get_symbol_ticker(symbol=symbol)
        current_price = float(current_prices['price'])
        return current_price
    except BinanceAPIException as e:
        # Handle Binance API exceptions
        print(f"API Exception occurred: {e.status_code} - {e.message}")
        return None
    except Exception as e:
        # Handle other unexpected errors
        print(f"An unexpected error occurred: {e}")
        return None

The present value of a specified cryptocurrency symbol (BTC) can be obtained by this function from the Binance API and a rate limitations is implemented to regulate the frequency of API requests. It correctly manages unforeseen problems and Binance API exceptions.

## Prediction Example

In [56]:
# Get current price from the Binance API
current_price = get_current_price(symbol)

# Predict next price using the loaded model
latest_prices_scaled = prices_scaled[-4:].reshape(1, 4, 1)

# Predict next price using the loaded model
predicted_price_scaled = loaded_model.predict(latest_prices_scaled)

# Inverse transform the predicted price to get the actual price
predicted_price = scaler.inverse_transform(predicted_price_scaled.reshape(-1, 1))[0][0]

# Print the current and predicted prices
print(f"Current Price: {current_price}, Predicted Price: {predicted_price}")

1/1 [==============================] - 0s 34ms/step
Current Price: 69050.62, Predicted Price: 69460.421875


# Tarding Logic And Actual Trading

In [ ]:
# Trading Logic And Actual Trading
symbol = "BTCUSDT"  # Symbol of Pair
quantity = '0.05'  # Quantity to trade
order = False


while True:
    try:
        # Fetch recent trades for the current price
        recent_trades = client.get_recent_trades(symbol=symbol)
        current_price = float(recent_trades[-1]['price']) 
        
        # Fetch candle data for prediction
        candles = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_15MINUTE, limit=4)
        model_data = np.array([float(candle[4]) for candle in candles]).reshape(-1, 1)
        model_data_scaled = scaler.transform(model_data).reshape(1, 4, 1)
        
        # Predict the next price
        predicted_price_scaled = loaded_model.predict(model_data_scaled)
        predicted_price = scaler.inverse_transform(predicted_price_scaled.reshape(-1, 1))[0][0]
        
        print(f"Current Price: {current_price}, Predicted Price: {predicted_price}")
        
        # Trading decision based on the prediction
        if not order and predicted_price > current_price:#
            order = True
            # Uncomment to place a market buy order
            #buy_price = client.get_order_book(symbol=symbol)['asks'][0][0] 
            #buy_order = client.order_market_buy(symbol=symbol, quantity=quantity)
            print(f"Buy @Market Price: {current_price}, Timestamp: {datetime.now()}")
            
        elif order and current_price > predicted_price:
            order = False
            # Uncomment to place a market sell order
            #sell_price = client.get_order_book(symbol=symbol)['bids'][0][0] 
            #sell_order = client.order_market_sell(symbol=symbol, quantity=quantity)
            print(f"Sell @Market Price: {current_price}, Timestamp: {datetime.now()}")
            

    except BinanceAPIException as e:
        print(f"Binance API Exception: {e.message}")
    except BinanceOrderException as e:
        print(f"Binance Order Exception: {e.message}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    time.sleep(60)  # Wait before the next iteration to avoid hitting rate limits

1/1 [==============================] - 0s 24ms/step
Current Price: 69050.63, Predicted Price: 69460.421875
Buy @Market Price: 69050.63, Timestamp: 2024-04-09 15:57:53.335300
1/1 [==============================] - 0s 33ms/step
Current Price: 69228.42, Predicted Price: 69525.2421875
1/1 [==============================] - 0s 49ms/step
Current Price: 69326.94, Predicted Price: 69561.8515625
1/1 [==============================] - 0s 34ms/step
Current Price: 69312.01, Predicted Price: 69418.84375
1/1 [==============================] - 0s 21ms/step
Current Price: 69172.0, Predicted Price: 69365.375
1/1 [==============================] - 0s 29ms/step
Current Price: 69192.01, Predicted Price: 69373.046875
1/1 [==============================] - 0s 24ms/step
Current Price: 69154.37, Predicted Price: 69358.6015625
1/1 [==============================] - 0s 16ms/step
Current Price: 69084.0, Predicted Price: 69333.015625
1/1 [==============================] - 0s 35ms/step
Current Price: 69042.77, Pre

A basic trading strategy based on price forecasts is demonstrated in this code section. It runs in a loop, regularly retrieving information from candlesticks and recent transaction data to predict the next change in price. It predicts the direction of price movement and then decides whether to execute a market buy or sell order for BTC Price. A comparison between the predicted cost and the current market price helps the process of decision-making. All noticed errors are appropriately handled to guarantee seamless operation, especially order exceptions and API problems. The script stops briefly in between iterations to avoid going above rate limitations.

## References


- Project on the Development of a Crypto Trading Bot. Detailed guide on creating and managing a cryptocurrency trading bot using Python and Binance API. Available at: https://dev.to/nicolasbonnici/how-to-build-a-crypto-bot-with-python-3-and-the-binance-api-part-1-1864
- Application of Technical Analysis Strategies in Cryptocurrency Trading. Explanation and implementation of technical trading strategies using the Binance API. Available at: https://www.learndatasci.com/tutorials/algo-trading-crypto-bot-python-strategy-backtesting/

- Handling Errors and API Management in Crypto Trading. Discussion on robust error handling techniques for maintaining operational stability of trading bots on Binance. Available at: https://www.learndatasci.com/tutorials/algo-trading-crypto-bot-python-strategy-backtesting/

- Binance Trading API – Creating Your First Simple Crypto-Trading Bot. Available at: https://blog.finxter.com/binance-api-trading-bot/

- Binance API documentation. https://binance-docs.github.io/apidocs/spot/en/

- Using LSTM for Cryptocurrency Trading. A detailed look at using LSTM models for predicting cryptocurrency prices effectively. Available at: https://www.analyticsvidhya.com/blog/2021/01/algotrading-using-technical-indicator-and-ml-models/

- Al-Nefaie, A.H. and Aldhyani, T.H.H., 2022. Bitcoin Price Forecasting and Trading: Data Analytics Approaches. Electronics, 11(24), p.4088. Available at: https://www.mdpi.com/2079-9292/11/24/4088 